In [1]:
using Pkg
Pkg.activate("$(@__DIR__)/..")

  Activating project at `~/hs2024/pdeongpu/LatticeBoltzmann.jl`


In [2]:
using ParallelStencil
@init_parallel_stencil(Threads, Float64, 3, inbounds=true)

In [3]:
const method = :D3Q19
include("../src/LatticeBoltzmann3D.jl")

thermal_convection_lbm_3D

In [4]:
init_global_grid(3, 3, 3)
finalize_global_grid(finalize_MPI=false)

Global grid: 3x3x3 (nprocs: 1, dims: 1x1x1; device support: none)


# Rayleigh Number Comparison

In [7]:
for Ra in [10, 100, 1000]
    thermal_convection_lbm_3D(N=40, nt=10000, Ra=Ra, vis_file="ra$Ra.mp4", manage_MPI=false)
end

# Parallel Efficiency Plot

In [6]:
using Plots

n_nodes = [1, 2, 4, 8, 27, 64]
t_effs = [291.65997961715567, 285.5423201147124, 285.9032788175847, 276.3592963556997, 275.9834819528713, 276.6327437563446]

# Assume ideal throughput is the same as the throughput for the smallest number of nodes
t_ideal = t_effs[1]  # Ideal performance for weak scaling

# Compute efficiency
efficiency =  t_effs./t_ideal

# Create the plot
plot(n_nodes, efficiency, label="Efficiency", xlabel="Number of Nodes", ylabel="Efficiency", marker=:circle, line=:auto)

# Save and show plot
savefig("../docs/plots/weak_scaling.png")

"/home/jonas/eth/HS 2024/pdeongpu/LatticeBoltzmann.jl/docs/plots/weak_scaling.png"

# 3D Contour Plot of Evolution

In [ ]:
using GLMakie, Plots, ProgressMeter

function load_array!(Aname, A)
    fname = string(Aname, ".bin")
    fid=open(fname, "r"); read!(fid, A); close(fid)
end

function contour_plot()
    lx, ly, lz = 40.0, 20.0, 24.0
    nx, ny, nz = 40, 20, 40
    temperature = zeros(Float32, nx - 2, ny - 2, nz - 2)

    visdir = "visdir"
    frames = 1:1001

    @showprogress for iframe in frames
        println(iframe)
        fig = Figure(size=(1600, 1000), fontsize=24)
        ax = Axis3(fig[1, 1]; aspect=(1, 1, 0.5), title="Temperature", xlabel="x", ylabel="z", zlabel="x")
        load_array!("$visdir/out_temp_$(lpad(iframe, 4, "0"))", temperature)
        @show size(temperature)
        surf = GLMakie.contour!(ax, permutedims(temperature, (1, 3, 2)); alpha=0.05, colormap=:turbo, levels=4)
        save("$visdir/$(lpad(iframe, 6, "0")).png", fig)
    end

    fnames = ["$(lpad(iframe, 6, "0"))" for iframe in frames]
    anim = Animation(visdir, fnames);
    gif(anim, "../docs/plots/3D_MULTI_XPU_surf.mp4", fps=24)
end

contour_plot()

# 3D Slice Plot of Evolution

In [ ]:
using GLMakie, Plots, ProgressMeter

function load_array!(Aname, A)
    fname = string(Aname, ".bin")
    fid=open(fname, "r"); read!(fid, A); close(fid)
end

function slice_plot()

    lx, ly, lz = 40.0, 20.0, 24.0
    nx, ny, nz = 200, 100, 200
    temperature = zeros(Float32, nx - 2, ny - 2, nz - 2)

    x = LinRange(0, lx, nx - 2)
    y = LinRange(0, ly, ny - 2)
    z = LinRange(0, lz, nz - 2)

    visdir = "visdir_200"
    frames = 1:1001

    @showprogress for iframe in frames
        fig = Figure(size=(1600, 1000), fontsize=24)
        ax = Axis3(fig[1, 1]; aspect=(1, 1, 0.5), title="Temperature", xlabel="x", ylabel="z", zlabel="y")
        load_array!("$visdir/out_temp_$(lpad(iframe, 4, "0"))", temperature)
        plt = volumeslices!(ax, x, z, y, permutedims(temperature, (1, 3, 2)); colormap=:turbo, alpha=0.8, colorrange=(-0.5, 0.5), bbox_visible=false)  

        plt[:update_yz][](Int((nx-2)/2))
        plt[:update_xz][](Int((nz-2)/2))
        plt[:update_xy][](Int(floor((ny-2)/5)))    

        save("$visdir/$(lpad(iframe, 6, "0")).png", fig)
    end

    fnames = ["$(lpad(iframe, 6, "0"))" for iframe in frames]
    anim = Animation(visdir, fnames);
    gif(anim, "../docs/plots/3D_MULTI_XPU_slice_200.mp4", fps=24)
end

slice_plot()